# Reddit NLP and Classification - Data Cleaning

### Import Libraries

In [9]:
import pandas as pd
import numpy as np

import re
import string
from html import unescape
import spacy


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# this setting widens how many characters pandas will display in a column:
pd.options.display.max_colwidth = 400

### Read Data

We will be using the PlayStation and Xbox csv files provided and check the first rows of the data to make sure the data was read correctly.

In [10]:
# import data for subreddit 1 and subreddit 2
ps = pd.read_csv('../data/playstation.csv')
xb = pd.read_csv('../data/xbox.csv')

In [11]:
ps.head(2)

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url
0,playstation,jc4b34,Blix404,8,PlayStation. Why\n1st of PlayStation has been out since 2013 and you guys close to releasing the PS5 so why tf do you now all of a sudden start recording our party conversations.\n2nd of I ain’t getting banned just cause some snowflake in my party chat felt the need to snitch on me. Fuck outta here with that BS.\n\n3rd. Xbox Doesn’t do that so why do you feel the need to do that you dickheads....,PARTY CHAT BEING RECORDED,1.0,https://www.reddit.com/r/playstation/comments/jc4b34/party_chat_being_recorded/
1,playstation,jc4070,ImmaPoodle,0,"So watch dogs legion is coming out soon and with interesting mechanics and ray tracing I think it might be what people thought watch dogs 2 would be... a predecessor to GTA 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that GTA had as well as the already large player base.\n\nBut this time with already tons of preordered copies of watch dogs legion in circle...",Ubisoft vs Rockstar,1.0,https://www.reddit.com/r/playstation/comments/jc4070/ubisoft_vs_rockstar/


In [12]:
xb.head(2)

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url
0,xbox,jc40iu,ImmaPoodle,1,"So watch dogs legion is coming out soon and with interesting mechanics and ray tracing I think it might be what people thought watch dogs 2 would be... a predecessor to GTA 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that GTA had as well as the already large player base.\n\nBut this time with already tons of preordered copies of watch dogs legion in circle...",Ubisoft vs Rockstar,1.0,https://www.reddit.com/r/xbox/comments/jc40iu/ubisoft_vs_rockstar/
1,xbox,jc3ty3,jaaytf_,2,"hey so my xbox one is turning on and instantly turning off. it’s so fast the sound of it coming on sounds cut off. i tried using the hairdryer technique, i borrowed a friends power brick, and nothing has worked. is there any last chance things i can do to fix it before i fully get rid of it bc it’s broken? \n\nalso, if it is 100% broken for some random reason, is there anything useful i can sa...",xbox one problems:),1.0,https://www.reddit.com/r/xbox/comments/jc3ty3/xbox_one_problems/


In [13]:
ps.shape, xb.shape

((10021, 8), (10005, 8))

In [14]:
ps.columns

Index(['subreddit', 'id', 'author', 'num_comments', 'selftext', 'title',
       'upvote_ratio', 'url'],
      dtype='object')

In [15]:
xb.columns

Index(['subreddit', 'id', 'author', 'num_comments', 'selftext', 'title',
       'upvote_ratio', 'url'],
      dtype='object')

In [16]:
ps.dtypes

subreddit        object
id               object
author           object
num_comments      int64
selftext         object
title            object
upvote_ratio    float64
url              object
dtype: object

#### Null values

Find and treat null values

In [17]:
ps.isnull().sum()[ps.isnull().sum() > 0]

selftext    83
dtype: int64

In [18]:
xb.isnull().sum()[xb.isnull().sum() > 0]

selftext    888
dtype: int64

Given the volume of data that is present here, I'm going to drop the rows with missing values.

In [19]:
ps.dropna(inplace=True)

In [20]:
xb.dropna(inplace=True)

### Additional Cleaning Approach

Things that I would like to address in cleaning (many derived from the class brainstorm):
- Duplicated or deleted/redacted posts
- Combine selftext and title so only treating one feature
- Make all lower case and strip leading/trailing spaces
- URLs
- Encoded characters
- Lemmatization
- Punctuation
- Extra spaces
- Numerics
- Keywords (playstation, xbox, ps3, ps4, ps5, sony, microsoft, etc.) to not 'cheat' on the project


After cleaning, during EDA, there may be additional items identified that will be added to this list, but this should be a good starting point.

> **NOTE** Some sampling was used throughout the steps below to check text to identify any concerns and see how various operations worked.  That code is commented out to make the notebook more digestible.

In [21]:
# check count of observations after dropping records
ps.shape[0], xb.shape[0]

(9938, 9117)

#### Check for and clean deleted and duplicated posts

Look for duplicates in the data, and treat if needed.

In [22]:
ps.duplicated(['selftext', 'title']).sum()

39

Take a look at what is duplicated to see if there are any interesting insights or patterns.

In [23]:
# ps[ps.duplicated(['selftext', 'title'])]

There was at least one post that appears as `[deleted]`, so let's check for that before we start dropping any data.

Also, an example post that has a few elements illustrated that we might want to clean like the encoded characters, currency symbols, etc.:

Are the almost +45€ worth it? (from around 120€ to 154€)\n\nPlanning on using it with the Logitech G29 + gear shifter (which I'll only buy once I also get the steering wheel stand).\n\nNot a heavy racing player, just looking for a better experience when playing racing games.\n\n&amp;#x200B;\n\nAlso: I'm open for other suggestionAre the almost +45€ worth it? (from around 120€ to 154€)\n\nPlanning on using it with the Logitech G29 + gear shifter (which I'll only buy once I also get the steering wheel stand).\n\nNot a heavy racing player, just looking for a better experience when playing racing games.\n\n&amp;#x200B;\n\nAlso: I'm open for other suggestion

- `&#x200B;` apparently is hex-encoded whitespace (ref: https://www.reddit.com/r/OutOfTheLoop/comments/9abjhm/what_does_x200b_mean/)
- `&amp` appears to be an HTML entity.  The list for HTML 4 is longer, but HTML 2 is: `&quot;, &amp;, &lt;, and &gt`.  May start with these and then expand if needed. (ref: https://www.htmlhelp.com/reference/html40/entities/special.html)
- `\n` lots of newline escape sequences as well.

Investigate deleted

In [24]:
# find how many posts have selftext set to [deleted]
len(ps[ps['selftext'] == '[deleted]'])

125

In [25]:
# check to see if there are deleted titles as well
len(ps[ps['title'] == '[deleted]'])

0

In [26]:
# check Xbox data for deleted as well
len(xb[xb['selftext'] == '[deleted]'])

200

In [27]:
# check Xbox titles for deleted
len(xb[xb['title'] == '[deleted]'])

0

Drop rows with `[deleted]`selftext

In [28]:
ps.drop(ps[ps['selftext'] == '[deleted]'].index, inplace=True)
xb.drop(xb[xb['selftext'] == '[deleted]'].index, inplace=True)

Drop duplicate rows

In [29]:
ps.drop_duplicates(['selftext', 'title'], inplace=True)
xb.drop_duplicates(['selftext', 'title'], inplace=True)

#### Combine selftext and title so only treating one feature

I will create a single text column that includes the words from the title and the selftext, so that additional cleaning functions will only need to operate on one feature.

Leaving selftext and title features for possible use in EDA, but will likely drop before any NLP processing.

In [30]:
ps['text'] = ps['title'] + ' ' + ps['selftext']
xb['text'] = xb['title'] + ' ' + xb['selftext']

In [31]:
ps.head(2)

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url,text
0,playstation,jc4b34,Blix404,8,PlayStation. Why\n1st of PlayStation has been out since 2013 and you guys close to releasing the PS5 so why tf do you now all of a sudden start recording our party conversations.\n2nd of I ain’t getting banned just cause some snowflake in my party chat felt the need to snitch on me. Fuck outta here with that BS.\n\n3rd. Xbox Doesn’t do that so why do you feel the need to do that you dickheads....,PARTY CHAT BEING RECORDED,1.0,https://www.reddit.com/r/playstation/comments/jc4b34/party_chat_being_recorded/,PARTY CHAT BEING RECORDED PlayStation. Why\n1st of PlayStation has been out since 2013 and you guys close to releasing the PS5 so why tf do you now all of a sudden start recording our party conversations.\n2nd of I ain’t getting banned just cause some snowflake in my party chat felt the need to snitch on me. Fuck outta here with that BS.\n\n3rd. Xbox Doesn’t do that so why do you feel the need...
1,playstation,jc4070,ImmaPoodle,0,"So watch dogs legion is coming out soon and with interesting mechanics and ray tracing I think it might be what people thought watch dogs 2 would be... a predecessor to GTA 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that GTA had as well as the already large player base.\n\nBut this time with already tons of preordered copies of watch dogs legion in circle...",Ubisoft vs Rockstar,1.0,https://www.reddit.com/r/playstation/comments/jc4070/ubisoft_vs_rockstar/,"Ubisoft vs Rockstar So watch dogs legion is coming out soon and with interesting mechanics and ray tracing I think it might be what people thought watch dogs 2 would be... a predecessor to GTA 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that GTA had as well as the already large player base.\n\nBut this time with already tons of preordered copies of watch d..."


#### Make all lower case and strip leading/trailing spaces

In [32]:
# create function that can be re-used or added to a class later
# may want to augment to accept df names and column name(s) to 
# make it more robust

def lcase_ss(text):
    '''
    fn returns all lower case text with spaces stripped (.lower() .strip())
    '''
    return text.lower().strip()

In [33]:
ps['text'] = ps['text'].apply(lcase_ss)
xb['text'] = xb['text'].apply(lcase_ss)

In [34]:
xb.head(2)

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url,text
0,xbox,jc40iu,ImmaPoodle,1,"So watch dogs legion is coming out soon and with interesting mechanics and ray tracing I think it might be what people thought watch dogs 2 would be... a predecessor to GTA 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that GTA had as well as the already large player base.\n\nBut this time with already tons of preordered copies of watch dogs legion in circle...",Ubisoft vs Rockstar,1.0,https://www.reddit.com/r/xbox/comments/jc40iu/ubisoft_vs_rockstar/,"ubisoft vs rockstar so watch dogs legion is coming out soon and with interesting mechanics and ray tracing i think it might be what people thought watch dogs 2 would be... a predecessor to gta 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that gta had as well as the already large player base.\n\nbut this time with already tons of preordered copies of watch d..."
1,xbox,jc3ty3,jaaytf_,2,"hey so my xbox one is turning on and instantly turning off. it’s so fast the sound of it coming on sounds cut off. i tried using the hairdryer technique, i borrowed a friends power brick, and nothing has worked. is there any last chance things i can do to fix it before i fully get rid of it bc it’s broken? \n\nalso, if it is 100% broken for some random reason, is there anything useful i can sa...",xbox one problems:),1.0,https://www.reddit.com/r/xbox/comments/jc3ty3/xbox_one_problems/,"xbox one problems:) hey so my xbox one is turning on and instantly turning off. it’s so fast the sound of it coming on sounds cut off. i tried using the hairdryer technique, i borrowed a friends power brick, and nothing has worked. is there any last chance things i can do to fix it before i fully get rid of it bc it’s broken? \n\nalso, if it is 100% broken for some random reason, is there anyt..."


#### URLs
In order to be able to remove URLs effectively, I think they should be removed before additional processing.   

In [35]:
# found URL regex at : https://urlregex.com
url_p = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# check to see if any matches in the text
ps['text'].str.contains(url_p).sum(), xb['text'].str.contains(url_p).sum()

(365, 393)

Since that pattern seemed to identify some URLs in the text, I will remove them

In [36]:
ps['text'] = ps['text'].str.replace(url_p, '', regex=True)
xb['text'] = xb['text'].str.replace(url_p, '', regex=True)

In [37]:
# check that worked
ps['text'].str.contains(url_p).sum(), xb['text'].str.contains(url_p).sum()

(0, 0)

#### Encoded Characters

Again, like URLs, we will want to process any encoded characters, specifically, I spotted some of the HTML encoded characters, so we'll tackle those first and can come back and change this later if we need to.

In [38]:
# https://stackoverflow.com/questions/26127775/remove-html-entities-and-extract-text-content-using-regex
# in that same stackoverflow, another user points to an unescape function in 
# html python library, that is what I ended up using

In [39]:
# get rid of html escape characters
ps['text'] = ps['text'].apply(unescape)
xb['text'] = xb['text'].apply(unescape)

While thinking about encoded characters and such, I will also remove `&#x200B;` and `\n` here

In [40]:
ps['text'] = ps['text'].str.replace('\n', ' ', regex=True)
xb['text'] = xb['text'].str.replace('\n', ' ', regex=True)

In [41]:
ps['text'] = ps['text'].str.replace('\u200B', ' ', regex=True)
xb['text'] = xb['text'].str.replace('\u200B', ' ', regex=True)

#### Lemmatization

Before moving forward with some of the other cleaning processes, I'll lemmatize at this point and also have Spacy remove punctuation.

In [42]:
# load spacy model
nlp = spacy.load('en_core_web_md')

In [43]:
# from spacy walkthrough
# ' '.join([token.lemma_ for token in ex if token.pos_ not in ['AUX', 'PUNCT', "PRON"]])

def spacy_lem(text):
    # create spacy doc
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.pos_ != 'PUNCT'])

In [44]:
# lemmatize and remove punctuation from first dataframe
ps['text'] = ps['text'].apply(spacy_lem)

In [45]:
# ps['text'].sample(20)

In [46]:
# lemmatize and remove punctuation from second dataframe
xb['text'] = xb['text'].apply(spacy_lem)

In [47]:
# xb['text'].sample(20)

#### Punctuation

The spacy process above cleared most of the punctutation, but from some of the samples I inspected above, it looked like some punctuation is still present.  Let's remove common punctuation marks from text.

In [48]:
# neat little regex thing from 
# https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
punc_p = re.compile('[%s]' % re.escape(string.punctuation))

In [49]:
# replacing with a space, as I saw later on that some things, like if they were
# separated by / would be confusing to discern
ps['text'] = ps['text'].str.replace(punc_p, ' ', regex=True)
xb['text'] = xb['text'].str.replace(punc_p, ' ', regex=True)

In [50]:
# ps['text'].sample(20)

#### Extra Spaces
Remove all extra spaces from the text

In [51]:
# check for extra spaces
ps['text'].str.contains('\s\s+').sum(), xb['text'].str.contains('\s\s+').sum()

(5212, 4645)

In [52]:
ps['text'] = ps['text'].str.replace('\s\s+', ' ', regex=True)
xb['text'] = xb['text'].str.replace('\s\s+', ' ', regex=True)

#### Numerics
I'd like to remove numbers that don't have additional letters trailing them, so that we can preserve things like 1080p or 4K.

In [53]:
ps['text'].str.contains(r'\b\d+\b').sum(), xb['text'].str.contains(r'\b\d+\b').sum()

(4282, 4196)

In [54]:
ps['text'] = ps['text'].str.replace(r'\b\d+\b', '', regex=True)
xb['text'] = xb['text'].str.replace(r'\b\d+\b', '', regex=True)

In [55]:
ps['text'].str.contains(r'\b\d+\b').sum(), xb['text'].str.contains(r'\b\d+\b').sum()

(0, 0)

In [56]:
# ps['text'].sample(20)

#### Keyword Removal

Create a list of content specific keywords to remove.  Having terms like playstation, sony, microsoft, and xbox prevalent in posts pertaining to one platform or the other is not advised for this project.

In [57]:
kwds = ['playstation', 'xbox', 'ps', 'ps2', 'ps3', 'ps4', 'ps5', 'sony', 'microsoft']

for kw in kwds:
    ps['text'] = ps['text'].str.replace(kw, '', regex=True)
    xb['text'] = xb['text'].str.replace(kw, '', regex=True)

In [58]:
ps['text'].str.contains('playstation').sum()

0

### Final Checks, Merge, Export

- Check shapes of DataFrames to make sure the target distribution will be similar for modeling.
- Drop any remaining features that are not needed.
- Merge the two DataFrames.
- Export a single, cleaned, CSV for EDA, preprocessing, modeling.


In [59]:
ps.shape, xb.shape

((9775, 9), (8889, 9))

In [60]:
ps.columns, xb.columns

(Index(['subreddit', 'id', 'author', 'num_comments', 'selftext', 'title',
        'upvote_ratio', 'url', 'text'],
       dtype='object'),
 Index(['subreddit', 'id', 'author', 'num_comments', 'selftext', 'title',
        'upvote_ratio', 'url', 'text'],
       dtype='object'))

In [61]:
ps.head(2)

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url,text
0,playstation,jc4b34,Blix404,8,PlayStation. Why\n1st of PlayStation has been out since 2013 and you guys close to releasing the PS5 so why tf do you now all of a sudden start recording our party conversations.\n2nd of I ain’t getting banned just cause some snowflake in my party chat felt the need to snitch on me. Fuck outta here with that BS.\n\n3rd. Xbox Doesn’t do that so why do you feel the need to do that you dickheads....,PARTY CHAT BEING RECORDED,1.0,https://www.reddit.com/r/playstation/comments/jc4b34/party_chat_being_recorded/,party chat be record why 1st of have be out since and you guy close to release the 5 so why tf do you now all of a sudden start record our party conversation 2nd of I ai not getting ban just cause some snowflake in my party chat feel the need to snitch on I fuck outta here with that bs 3rd do not do that so why do you feel the need to do that you dickhead just prove to be more and more of ...
1,playstation,jc4070,ImmaPoodle,0,"So watch dogs legion is coming out soon and with interesting mechanics and ray tracing I think it might be what people thought watch dogs 2 would be... a predecessor to GTA 5, ok let's be honest watch dogs 2 was an amazing game but it lacked that feeling that GTA had as well as the already large player base.\n\nBut this time with already tons of preordered copies of watch dogs legion in circle...",Ubisoft vs Rockstar,1.0,https://www.reddit.com/r/playstation/comments/jc4070/ubisoft_vs_rockstar/,ubisoft vs rockstar so watch dog legion be come out soon and with interesting mechanic and ray trace I think it might be what people think watch dog would be a predecessor to gta ok let us be honest watch dog be an amazing game but it lack that feeling that gta have as well as the already large player base but this time with already ton of preordere copy of watch dog legion in circle and ro...


As this classification is focused around posts, titles, and comments, and the cleaning and duplicate checks have already been performed, the following features will be removed:
- id
- url
- upvote ratio

I'll leave the remaining columns as they might be used for EDA.

In [62]:
drop = ['id', 'url', 'upvote_ratio']

ps.drop(columns=drop, inplace=True)
xb.drop(columns=drop, inplace=True)

#### Merge and Export

In [63]:
# check final shapes
ps.shape, xb.shape

((9775, 6), (8889, 6))

In [64]:
# diff the columns
set(ps.columns) - set(xb.columns)

set()

In [65]:
# create a combined data frame
clean_df = pd.concat([ps, xb])

In [66]:
# check the shape of the combined dataframe
clean_df.shape

(18664, 6)

In [67]:
# export the dataframe as csv
clean_df.to_csv('../data/reddit_cleaned.csv', index=False)